In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('updated_capstonedata.csv', encoding='latin1', error_bad_lines=False, sep=';')

# Filtering system for food allergies
!pip install pandas openpyxl

import pandas as pd

allergies = ["alcoholic", "almond", "peanut", "egg", "milk", "wheat", "soy", "chicken", "beef", "cashew", "cheese", "clam", "shrimp", "coconut", "dairy", "fish", "pork", "seafood", "squid", "crab", "strawberry"]

def filter_allergies(user_input):
    filtered_allergies = []
    for allergy in allergies:
        if allergy in user_input.lower():
            filtered_allergies.append(allergy)
    return filtered_allergies

 # Prompt the user to enter their allergies
filtered_allergies = input("Enter your allergies: ")
# Split the data into features (X) and target (y)
X = data.drop(['Recipe_Name', 'rating'], axis=1).values
y = data['rating'].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

<ipython-input-2-6dd073d1ca3c>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('updated_capstonedata.csv', encoding='latin1', error_bad_lines=False, sep=';')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Enter your allergies: shrimp


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [4]:
model.compile(optimizer='adam', loss='mse')

In [5]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

Epoch 1/100
243/243 [==============================] - 3s 5ms/step - loss: 13.5261 - val_loss: 4324.8760
Epoch 2/100
243/243 [==============================] - 2s 8ms/step - loss: 5.0639 - val_loss: 19.0212
Epoch 3/100
243/243 [==============================] - 3s 13ms/step - loss: 2.7223 - val_loss: 54.4912
Epoch 4/100
243/243 [==============================] - 2s 8ms/step - loss: 1.9659 - val_loss: 1.9822
Epoch 5/100
243/243 [==============================] - 2s 9ms/step - loss: 1.9332 - val_loss: 11.2186
Epoch 6/100
243/243 [==============================] - 2s 9ms/step - loss: 0.9461 - val_loss: 1.5229
Epoch 7/100
243/243 [==============================] - 2s 9ms/step - loss: 1.6052 - val_loss: 1.4967
Epoch 8/100
243/243 [==============================] - 3s 10ms/step - loss: 0.9169 - val_loss: 0.8472
Epoch 9/100
243/243 [==============================] - 3s 10ms/step - loss: 1.2510 - val_loss: 1.6088
Epoch 10/100
243/243 [==============================] - 2s 6ms/step - loss: 1.090

In [6]:
# Get the user input for recipe name and rating
recipe_name = input("Enter the recipe name: ")
rating = float(input("Rate the recipe (1-5): "))

# Find the index of the recipe in the dataset
recipe_index = data[data['Recipe_Name'] == recipe_name].index[0]

# Extract the features for the selected recipe
recipe_features = X[recipe_index]

# Calculate the Euclidean distances between features
distances = np.linalg.norm(X - recipe_features, axis=1)

# Sort the distances in ascending order
sorted_distances = np.argsort(distances)

# If the rating is high, recommend similar recipes
if rating >= 4:
    sorted_distances = sorted_distances[sorted_distances != recipe_index]

    recommended_recipes = sorted_distances[:5]  # Get top 5 similar recipes
else:
    sorted_distances = sorted_distances[sorted_distances != recipe_index]

    recommended_recipes = sorted_distances[-5:] # Get the top 5 dissimilar recipes

# Print the recommended recipe names
print("Recommended recipes:")
for index in recommended_recipes:
    recipe_name = data.loc[index, 'Recipe_Name']
    filtered_recipe_allergies = filter_allergies(recipe_name)
    if not filtered_recipe_allergies:
        print(recipe_name)

Enter the recipe name: Pumpkin Muffins 
Rate the recipe (1-5): 5
Recommended recipes:
Pumpkin Muffins 
Saffron Vichyssoise 
Double-Corn Chowder with Chipotle and Bacon 
Tomato, Arugula, and Ricotta Salata Salad 


In [7]:
import os

directory = './saved_model/'

if not os.path.exists(directory):
    os.makedirs(directory)

In [8]:
# Define the directory path to save the model
model_directory = './saved_model/'

# Save the model
tf.saved_model.save(model, model_directory)

print("Model saved at:", model_directory)

Model saved at: ./saved_model/


In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the saved model
loaded_model = tf.saved_model.load('./saved_model/')

allergies = ["alcoholic", "almond", "peanut", "egg", "milk", "wheat", "soy", "chicken", "beef", "cashew", "cheese", "clam", "shrimp", "coconut", "dairy", "fish", "pork", "seafood", "squid", "crab", "strawberry"]

# Function to filter recipes based on allergies
def filter_recipes(allergies, recipe_names):
    filtered_recipes = []
    for recipe_name in recipe_names:
        contains_allergen = False
        for allergy in allergies:
            if allergy in recipe_name.lower():
                contains_allergen = True
                break
        if not contains_allergen:
            filtered_recipes.append(recipe_name)
    return filtered_recipes

# Prompt the user to enter their allergies
print("Enter your allergies (separated by commas):")
filtered_allergies = input().lower().split(",")

# Get the user input for recipe name and rating
recipe_name = input("Enter the recipe name: ")
rating = float(input("Rate the recipe (1-5): "))

# Find the index of the recipe in the dataset
recipe_index = data[data['Recipe_Name'] == recipe_name].index[0]

# Extract the features for the selected recipe
recipe_features = X[recipe_index]

# Reshape the input data to match the expected shape of the model
input_data = np.expand_dims(recipe_features, axis=0)

# Make the prediction
signature = loaded_model.signatures['serving_default']
predicted_rating = signature(tf.constant(input_data.astype(np.float32)))

# Calculate the similarity scores between the selected recipe and all other recipes
similarity_scores = cosine_similarity(X, recipe_features.reshape(1, -1))

# Sort the similarity scores in descending order
sorted_indices = np.argsort(similarity_scores.ravel())[::-1]

# Get the top 5 indices (excluding the selected recipe itself)
top_indices = sorted_indices[1:6]

if rating <= 3:
    # Get the least similar indices (excluding the selected recipe itself)
    dissimilar_indices = sorted_indices[-6:-1]
    top_indices = dissimilar_indices

# Get the recipe names for the top indices
top_recipe_names = data.iloc[top_indices]['Recipe_Name'].values

# Filter recipes based on allergies
filtered_recipe_names = filter_recipes(filtered_allergies, top_recipe_names)

print("Recommended Recipes:")
for i, recipe in enumerate(filtered_recipe_names, 1):
    print(f"{i}. {recipe}")

Enter your allergies (separated by commas):
blueberry
Enter the recipe name: Pumpkin Muffins 
Rate the recipe (1-5): 5
Recommended Recipes:
1. Pumpkin Muffins 
2. Whole Grain Cranberry-Apple Scones 
3. Sweet-Potato Sticks with Mustard Seeds and Ginger 
4. Red Bell Pepper, Raisin and Olive Stuffing 


In [11]:
import tensorflow as tf
import numpy as np

# Load the saved model
loaded_model = tf.saved_model.load('./saved_model')

# Get the user input for recipe name and rating
recipe_name = input("Enter the recipe name: ")
rating = float(input("Rate the recipe (1-5): "))

# Find the index of the recipe in the dataset
recipe_index = data[data['Recipe_Name'] == recipe_name].index[0]

# Extract the features for the selected recipe
recipe_features = X[recipe_index]

# Reshape the input data to match the expected shape of the model
input_data = np.expand_dims(recipe_features, axis=0)

# Convert the input data to a tensor
input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)

# Make the prediction
predicted_rating = loaded_model(input_tensor)

print("Predicted Rating:", predicted_rating)

Enter the recipe name: Pumpkin Muffins 
Rate the recipe (1-5): 5
Predicted Rating: tf.Tensor([[4.0557623]], shape=(1, 1), dtype=float32)


In [12]:
preds = np.round(model.predict(X_val),0)
from sklearn.metrics import confusion_matrix
# confusion matrix
confusion_matrix(y_val, preds)

61/61 [==============================] - 0s 3ms/step


array([[   2,    0,    1,   18,    2],
       [   0,    2,    1,   12,    1],
       [   0,    2,   20,  209,    9],
       [   0,    2,   33, 1271,   47],
       [   0,    1,    9,  240,   60]])

In [13]:
# Calculate and print metrics
preds_train = np.round(model.predict(X_train), 0)
preds_val = np.round(model.predict(X_val), 0)

accuracy_train = np.mean(np.abs(preds_train - y_train) <= 1.0)
accuracy_val = np.mean(np.abs(preds_val - y_val) <= 1.0)

loss_train = model.evaluate(X_train, y_train, verbose=0)
loss_val = model.evaluate(X_val, y_val, verbose=0)

mse_train = np.mean((preds_train - y_train) ** 2)
mse_val = np.mean((preds_val - y_val) ** 2)

rmse_train = np.sqrt(mse_train)
rmse_val = np.sqrt(mse_val)

mae_train = np.mean(np.abs(preds_train - y_train))
mae_val = np.mean(np.abs(preds_val - y_val))

print("Accuracy (Train):", accuracy_train)
print("Accuracy (Validation):", accuracy_val)
print("Training Loss:", loss_train)
print("Validation Loss:", loss_val)
print("RMSE (Train):", rmse_train)
print("RMSE (Validation):", rmse_val)
print("MAE (Train):", mae_train)
print("MAE (Validation):", mae_val)


61/61 [==============================] - 0s 2ms/step
Accuracy (Train): 0.9653425888670195
Accuracy (Validation): 0.96335278164611
Training Loss: 0.22686663269996643
Validation Loss: 0.4264606535434723
RMSE (Train): 0.7211462149196699
RMSE (Validation): 0.7357004147591034
MAE (Train): 0.3915377755127619
MAE (Validation): 0.40917821774827356


In [ ]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    # Get the recipe name from the request
    recipe_name = request.json['recipe_name']

In [ ]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

# Define the route for prediction
@app.route('/predict', methods=['POST'])
def predict():
    # Get the recipe name from the request
    recipe_name = request.json['recipe_name']

    # Find the index of the recipe in the dataset
    recipe_index = data[data['Recipe_Name'] == recipe_name].index[0]

    # Extract the features for the selected recipe
    recipe_features = X[recipe_index]

    # Calculate the Euclidean distances between features
    distances = np.linalg.norm(X - recipe_features, axis=1)

    # Sort the distances in ascending order
    sorted_distances = np.argsort(distances)

    # Get the top 5 similar recipes
    recommended_recipes = sorted_distances[:5]

    # Get the recipe names
    recipe_names = []
    for index in recommended_recipes:
        recipe_name = data.loc[index, 'Recipe_Name']
        recipe_names.append(recipe_name)

    # Return the recipe names as the API response
    return jsonify({'recommended_recipes': recipe_names})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [14]:
import shutil

# Specify the path to the saved model directory
model_directory = './saved_model/'

# Specify the path for the zip file
zip_file_path = '/content/saved_model.zip'

# Zip the model directory
shutil.make_archive(zip_file_path.split('.')[0], 'zip', model_directory)

# Download the zip file
from google.colab import files
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>